# Reading data from SQL databases

So far we've only talked about reading data from CSV files. That's a pretty common way to store data, but there are many others! Pandas can read from HTML, JSON, SQL, Excel (!!!), HDF5, Stata, and a few other things. In this chapter we'll talk about reading data from SQL databases.

You can read data from a SQL database using the `pd.read_sql` function. `read_sql` will automatically convert SQL column names to DataFrame column names.

`read_sql` takes 2 arguments: a `SELECT` statement, and a database connection object. This is great because it means you can read from *any* kind of SQL database -- it doesn't matter if it's MySQL, SQLite, PostgreSQL, or something else.

This example reads from a SQLite database, but any other database would work the same way.

In [1]:
import pandas as pd
import sqlite3

In [12]:
con = sqlite3.connect("Data/weather_2012.sqlite")  # we are Conncet to the  Data base
con

In [20]:
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", con)  # Ddisplay the Tables in the Data base
print(table)

              name
0     weather_2012
1  sqlite_sequence


In [21]:
df = pd.read_sql("SELECT * from weather_2012 LIMIT 5", con) # Read the Table
df

,id,date_time,temp
0,1,2012-01-01 00:00:00,-1.8
1,2,2012-01-01 01:00:00,-1.8
2,3,2012-01-01 02:00:00,-1.8
3,4,2012-01-01 03:00:00,-1.5
4,5,2012-01-01 04:00:00,-1.5


In [19]:
df_seq = pd.read_sql("SELECT * from sqlite_sequence LIMIT 3", con)
df_seq

,name,seq
0,weather_2012,100


`read_sql` doesn't automatically set the primary key (`id`) to be the index of the dataframe. You can make it do that by adding an `index_col` argument to `read_sql`. 

If you've used `read_csv` a lot, you may have seen that it has an `index_col` argument as well. This one behaves the same way.

In [22]:
df = pd.read_sql("SELECT * from weather_2012", con, index_col='id')
df

,date_time,temp
id,,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8
4,2012-01-01 03:00:00,-1.5
5,2012-01-01 04:00:00,-1.5
...,...,...
96,2012-01-04 23:00:00,-9.6
97,2012-01-05 00:00:00,-8.8
98,2012-01-05 01:00:00,-7.5


In [23]:
df.head()

,date_time,temp
id,,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8
4,2012-01-01 03:00:00,-1.5
5,2012-01-01 04:00:00,-1.5


In [24]:
df.tail()

,date_time,temp
id,,
96,2012-01-04 23:00:00,-9.6
97,2012-01-05 00:00:00,-8.8
98,2012-01-05 01:00:00,-7.5
99,2012-01-05 02:00:00,-5.4
100,2012-01-05 03:00:00,-5.0


In [25]:
df.describe()

,temp
count,100.000000
mean,-6.841000
std,7.774487
min,-18.600000
25%,-14.825000
50%,-7.450000
75%,0.350000
max,5.300000


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1 to 100
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_time  100 non-null    object 
 1   temp       100 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.3+ KB


If you want your dataframe to be indexed by more than one column, you can give a list of columns to `index_col`:

In [27]:
df = pd.read_sql("SELECT * from weather_2012 LIMIT 3", con, 
                 index_col=['id', 'date_time'])
df

,,temp
id,date_time,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8


# Writing to a SQLite database

Pandas has a `write_frame` function which creates a database table from a dataframe. Right now this only works for SQLite databases. Let's use it to move our 2012 weather data into SQL.

You'll notice that this function is in `pd.io.sql`. There are a ton of useful functions for reading and writing various kind of data in `pd.io`, and it's worth spending some time exploring them. ([see the documentation!](http://pandas.pydata.org/pandas-docs/stable/io.html))

In [29]:
weather_df = pd.read_csv('Data/weather_2012.csv')
weather_df

,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,2012-01-01 03:00:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,2012-01-01 04:00:00,-1.5,-3.3,88,7,4.8,101.23,Fog
...,...,...,...,...,...,...,...,...
8779,2012-12-31 19:00:00,0.1,-2.7,81,30,9.7,100.13,Snow
8780,2012-12-31 20:00:00,0.2,-2.4,83,24,9.7,100.03,Snow
8781,2012-12-31 21:00:00,-0.5,-1.5,93,28,4.8,99.95,Snow
8782,2012-12-31 22:00:00,-0.2,-1.8,89,28,9.7,99.91,Snow


In [30]:
con = sqlite3.connect("Data/test_db.sqlite")
con

In [33]:
con.execute("DROP TABLE IF EXISTS weather_2012")  # Drop the Table

In [34]:
weather_df.to_sql("weather_2012", con) # Converting the Csv to SQl

c:\users\reddy\appdata\local\programs\python\python36\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


We can now read from the `weather_2012` table in  `test_db.sqlite`, and we see that we get the same data back:

In [35]:
con = sqlite3.connect("Data/test_db.sqlite")
df = pd.read_sql("SELECT * from weather_2012 LIMIT 3", con)
df

,index,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"


The nice thing about having your data in a database is that you can do arbitrary SQL queries. This is cool especially if you're more familiar with SQL. Here's an example of sorting by the Weather column:

In [36]:
con = sqlite3.connect("Data/test_db.sqlite")
df = pd.read_sql("SELECT * from weather_2012 ORDER BY Weather", con)
df

,index,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,67,2012-01-03 19:00:00,-16.9,-24.8,50,24,25.0,101.74,Clear
1,114,2012-01-05 18:00:00,-7.1,-14.4,56,11,25.0,100.71,Clear
2,115,2012-01-05 19:00:00,-9.2,-15.4,61,7,25.0,100.80,Clear
3,116,2012-01-05 20:00:00,-9.8,-15.7,62,9,25.0,100.83,Clear
4,117,2012-01-05 21:00:00,-9.0,-14.8,63,13,25.0,100.83,Clear
...,...,...,...,...,...,...,...,...,...
8779,6188,2012-09-14 20:00:00,16.5,15.8,96,22,16.1,101.06,"Thunderstorms,Rain Showers"
8780,4323,2012-06-29 03:00:00,19.5,16.1,81,7,9.7,99.71,"Thunderstorms,Rain Showers,Fog"
8781,4761,2012-07-17 09:00:00,22.9,21.3,91,17,9.7,99.84,"Thunderstorms,Rain Showers,Fog"
8782,5108,2012-07-31 20:00:00,22.4,18.7,80,35,9.7,100.64,"Thunderstorms,Rain Showers,Fog"


If you have a PostgreSQL database or MySQL database, reading from it works exactly the same way as reading from a SQLite database. You create a connection using `psycopg2.connect()` or `MySQLdb.connect()`, and then use

`pd.read_sql("SELECT whatever from your_table", con)`

# Connecting to other kinds of database

To connect to a MySQL database:

*Note: For these to work, you will need a working MySQL / PostgreSQL database, with the correct localhost, database name, etc.*

To connect to a PostgreSQL database:

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  